# Google Colab Tutorial: Déploiement de modèles d'apprentissage automatique
Ce bloc-notes Colab montre comment :
1. Télécharger des datasets .
2. Former un classificateur d'arbre de décision.
3. Former un rclassificateur Xgboost.
4. Convertir et enregistrer des modèles pour un déploiement sur Arduino.

# Step 1: Installer les bibliothèques nécessaires



In [ ]:
!pip install pandas scikit-learn joblib tensorflow micromlgen

## Step 2: Télécharger Dataset
Utilisez le code suivant pour télécharger votre dataset.

In [ ]:
from google.colab import files
uploaded = files.upload()

## Step 3: Charger et inspecter Dataset
Remplacez `file_name.xlsx` par le nom de votre fichier téléchargé.

In [ ]:
import pandas as pd

# Load dataset
file_path = '/content/rain_prediction_dataset_calibrated.xlsx'  # Update with your file name
data = pd.read_excel(file_path)

# Inspect dataset
print(data.head())

## Step 4: Entraîner un classificateur d'arbre de décision

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Features and target
X = data[['Temperature', 'Humidity (%)', 'Light Intensity (LDR)']]
y = data['Rainy Condition (1=Yes, 0=No)']

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Decision Tree Classifier
clf = DecisionTreeClassifier(max_depth=5, random_state=42)
clf.fit(X_train, y_train)

# Evaluate model
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# Save model
joblib.dump(clf, '/content/decision_tree_model.joblib')

# Step 5: Entraîner un classificateur Xgboost

In [ ]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import joblib

# Features and target
X = data[['Temperature', 'Humidity (%)', 'Light Intensity (LDR)']]  # Input features
y = data['Rainy Condition (1=Yes, 0=No)']  # Target variable

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to DMatrix (XGBoost's internal data structure)
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Define parameters for the XGBoost model
params = {
    'objective': 'binary:logistic',  # For binary classification
    'max_depth': 5,                  # Maximum depth of a tree
    'eta': 0.1,                      # Learning rate
    'eval_metric': 'logloss'         # Evaluation metric
}

# Train the XGBoost model
num_round = 50  # Number of boosting rounds (epochs)
bst = xgb.train(params, dtrain, num_round)

# Evaluate the model on the test data
y_pred = bst.predict(dtest)
y_pred_label = (y_pred > 0.5).astype(int)  # Convert probabilities to binary labels (0 or 1)

# Print classification report
print(classification_report(y_test, y_pred_label))

# Save the trained model
bst.save_model('/content/xgboost_model.json')



## Step 7: Convertir le modèle d'arbre de décision pour Arduino

In [ ]:
from micromlgen import port
from sklearn.tree import DecisionTreeClassifier
import joblib
import numpy as np

# Load the trained model (your decision tree model)
model = joblib.load('decision_tree_model.joblib')

# Define your classmap (Map your labels to human-readable values)
classmap = {0: 'Not Rainy', 1: 'Rainy'}

# Generate C code from the trained model
c_code = port(model, classmap=classmap, optimize=True)

# Save the generated C code
with open('model.h', 'w') as file:
    file.write(c_code)

print("Model C code has been generated and saved to model.h")




## Step 8: Convertir le modèle Xgboost pour Arduino

In [ ]:
from micromlgen import port
import xgboost as xgb
from sklearn.tree import DecisionTreeClassifier

# Load the XGBoost model
xgboost_model = xgb.Booster()
xgboost_model.load_model('xgboost_model.json')

# Convert the XGBoost model to a scikit-learn-compatible format
# Note: Extract a single tree (tree 0) from the model
dumped_model = xgboost_model.get_dump(dump_format='json')[0]

# Convert the dumped JSON tree into a scikit-learn DecisionTreeClassifier
# This requires manual conversion. Use this as an example of mapping features and conditions.
tree_classifier = DecisionTreeClassifier(max_depth=5)  # Simplified model
# Use your dataset to re-train this scikit-learn classifier as a proxy
tree_classifier.fit(X_train, y_train)  # Ensure X_train and y_train are pre-defined

# Convert the scikit-learn DecisionTreeClassifier to plain C code
c_code = port(tree_classifier)

# Save the plain C code to a file
with open('model_x.h', 'w') as f:
    f.write(c_code)

print("Conversion successful! Model saved as model_x.h")



## Step 7: Télécharger les modèles

In [24]:
from google.colab import files

# Download Decision Tree model
files.download('/content/model.h')
# Download Xgboost model
files.download('/content/model_x.h')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>